# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, fbeta_score, make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\g800575\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\g800575\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\g800575\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
#print (engine.table_names())
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)

    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Note our data is highly imbalanced (very few positive classes for most categories)
# So setting class_weight='balanced'
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
import warnings
warnings.filterwarnings("ignore")

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False,
            random_state=None, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pred = pipeline.predict(X_test)

In [8]:
for i in range(36):
    print(category_names[i])
    cat_Y_test = Y_test.iloc[:,i].values
    cat_Y_pred = Y_pred[:,i]
    print(classification_report(cat_Y_test, cat_Y_pred))

related
              precision    recall  f1-score   support

           0       0.62      0.51      0.56      1222
           1       0.86      0.90      0.88      3984

   micro avg       0.81      0.81      0.81      5206
   macro avg       0.74      0.71      0.72      5206
weighted avg       0.80      0.81      0.81      5206

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      4332
           1       0.77      0.46      0.58       874

   micro avg       0.89      0.89      0.89      5206
   macro avg       0.84      0.72      0.76      5206
weighted avg       0.88      0.89      0.87      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5182
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5134
           1       0.00      0.00      0.00        72

   micro avg       0.99      0.99      0.99      5206
   macro avg       0.49      0.50      0.50      5206
weighted avg       0.97      0.99      0.98      5206

other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4968
           1       0.25      0.00      0.01       238

   micro avg       0.95      0.95      0.95      5206
   macro avg       0.60      0.50      0.49      5206
weighted avg       0.92      0.95      0.93      5206

weather_related
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      3697
           1       0.83      0.62      0.71      1509

   micro avg       0.85      0.85      0.85      5206
   macro avg       0.84      0.78      0.80      5206
weighted avg       0.85      0.85    

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# The default scoring for MultiOutputClassifier considers a prediction correct
# only if all categories of a test sample are predicted correctly. This is not what we want here.

# In this particular disaster response case, we want the messages to be sent to as-many-as-possible
# relevant parties (so they can work on the problems). Hence the recall for each category is more important.
# So creating a new scorer for the grid search.

In [10]:
# Define a new scorer for grid search
scorer = make_scorer(fbeta_score, beta=2, average='micro')

In [11]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.5922313015599954, total= 2.0min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.5831194890891201, total= 2.1min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.5843166009011007, total= 1.8min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, score=0.5943967076258345, total= 3.5min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, score=0.5846361706234318, total= 3.7min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, score=0.5840071877807727, total= 3.5min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.610724396013495, total= 1.7min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 36.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False,
            random_state=None, verbose=0, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=2, average=micro), verbose=3)

In [13]:
Y_pred = cv.predict(X_test)

In [14]:
for i in range(len(category_names)):
    print(category_names[i])
    cat_Y_test = Y_test.iloc[:,i].values
    cat_Y_pred = Y_pred[:,i]

    accuracy = accuracy_score(cat_Y_test, cat_Y_pred)
    precision = precision_score(cat_Y_test, cat_Y_pred)
    recall = recall_score(cat_Y_test, cat_Y_pred)
    print(f'    Accuracy: {accuracy:.4f}    % Precision: {precision:.4f}    % Recall: {recall:.4f}')

related
    Accuracy: 0.8312    % Precision: 0.8668    % Recall: 0.9209
request
    Accuracy: 0.9032    % Precision: 0.7447    % Recall: 0.6442
offer
    Accuracy: 0.9952    % Precision: 0.0000    % Recall: 0.0000
aid_related
    Accuracy: 0.7843    % Precision: 0.7347    % Recall: 0.7632
medical_help
    Accuracy: 0.9241    % Precision: 0.5694    % Recall: 0.1976
medical_products
    Accuracy: 0.9547    % Precision: 0.7500    % Recall: 0.1489
search_and_rescue
    Accuracy: 0.9702    % Precision: 0.7368    % Recall: 0.0854
security
    Accuracy: 0.9787    % Precision: 0.0000    % Recall: 0.0000
military
    Accuracy: 0.9721    % Precision: 0.6301    % Recall: 0.2805
child_alone
    Accuracy: 1.0000    % Precision: 0.0000    % Recall: 0.0000
water
    Accuracy: 0.9575    % Precision: 0.8128    % Recall: 0.4497
food
    Accuracy: 0.9418    % Precision: 0.8097    % Recall: 0.6426
shelter
    Accuracy: 0.9366    % Precision: 0.7907    % Recall: 0.4250
clothing
    Accuracy: 0.9848    % Pr

In [15]:
cv.best_params_

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 50}

In [16]:
cv.best_score_

0.6028120215372157

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# Try to add some extra features
# https://www.quora.com/Natural-Language-Processing-What-are-the-possible-features-that-can-be-extracted-from-text
# https://www.kaggle.com/shaz13/feature-engineering-for-nlp-classification

# Compute word count
def count_words(data):
    """Compute number of words
    
    Args:
        data (Series or 1D-array): Text data.
    
    Returns:
        NumPy 1D-arrary: Word count values for each text.
    """
    return np.array([len(text.split()) for text in data]).reshape(-1, 1)

In [18]:
# Try Naive Bayes algorithms (which is fast to train)
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('wordcount', FunctionTransformer(count_words, validate=False))
    ])),
    ('clf', MultiOutputClassifier(BernoulliNB()))
])

In [19]:
parameters = {
    'features__text_pipeline__vect__ngram_range': [(1,1), (1,2)],
    'clf__estimator__alpha': [0.01, 0.1],
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, cv=3, verbose=3)

cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1), score=0.5763066778101263, total=  15.6s
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.3s remaining:    0.0s


[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1), score=0.5654775354524996, total=  15.4s
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   52.0s remaining:    0.0s


[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 1), score=0.5755063205110779, total=  15.2s
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2), score=0.5477719677162669, total=  16.8s
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2), score=0.5308442625259344, total=  16.8s
[CV] clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__alpha=0.01, features__text_pipeline__vect__ngram_range=(1, 2), score=0.5367213083989378, total=  17.0s
[CV] clf__estimator__alpha=0.1, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  clf__estimator__alpha=0.1, features__text_pipeline__vect__ngram_range=(1, 1), score=0.5925794312476651, total=  15.1s
[CV] clf__estimator__alpha=0.1, features_

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.4min finished


In [20]:
cv.best_params_

{'clf__estimator__alpha': 0.1,
 'features__text_pipeline__vect__ngram_range': (1, 1)}

In [21]:
cv.best_score_

0.5824769113413322

In [22]:
print('Custom fbeta score is {}'.format(fbeta_score(Y_test, Y_pred, beta=2, average='micro')))

for i in range(len(category_names)):
    print(category_names[i])
    cat_Y_test = Y_test.iloc[:,i].values
    cat_Y_pred = Y_pred[:,i]

    accuracy = accuracy_score(cat_Y_test, cat_Y_pred)
    precision = precision_score(cat_Y_test, cat_Y_pred)
    recall = recall_score(cat_Y_test, cat_Y_pred)
    print(f'    Accuracy: {accuracy:.4f}    % Precision: {precision:.4f}    % Recall: {recall:.4f}')

Custom fbeta score is 0.5943816176385127
related
    Accuracy: 0.7887    % Precision: 0.8970    % Recall: 0.8178
request
    Accuracy: 0.8225    % Precision: 0.4826    % Recall: 0.7918
offer
    Accuracy: 0.9937    % Precision: 0.0000    % Recall: 0.0000
aid_related
    Accuracy: 0.7516    % Precision: 0.7168    % Recall: 0.6779
medical_help
    Accuracy: 0.8955    % Precision: 0.3702    % Recall: 0.4434
medical_products
    Accuracy: 0.9280    % Precision: 0.3323    % Recall: 0.4275
search_and_rescue
    Accuracy: 0.9579    % Precision: 0.2650    % Recall: 0.1890
security
    Accuracy: 0.9697    % Precision: 0.1364    % Recall: 0.0818
military
    Accuracy: 0.9524    % Precision: 0.3521    % Recall: 0.6098
child_alone
    Accuracy: 1.0000    % Precision: 0.0000    % Recall: 0.0000
water
    Accuracy: 0.9351    % Precision: 0.5000    % Recall: 0.4911
food
    Accuracy: 0.9141    % Precision: 0.6200    % Recall: 0.6460
shelter
    Accuracy: 0.9024    % Precision: 0.4724    % Recall: 0.5

### 9. Export your model as a pickle file

In [23]:
model_filepath = 'models/classifier.pkl'
pickle.dump(cv, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.